In [ ]:
import torch
from transformers import GPT2Tokenizer, GPT2LMHeadModel
from mingpt.model import GPT
from mingpt.utils import set_seed
from mingpt.bpe import BPETokenizer
set_seed(3407)

In [ ]:
if use_mingpt:
  model = GPT.from_pretrained(model_type)
else:
  model = GPT2LMHeadModel.from_pretrained(model_type)
  model.config.pad_token_id = model.config.eos_token_id

model.to(device)
model.eval();

In [ ]:
def generate(prompt='', num_samples = 10, steps = 20, do_sample=True):

  if use_mingpt:
    tokenizer = BPETokenizer()
    if prompt == '':
      x = torch.tensor([[tokenizer.encoder.encoder['<|endoftext|>']]], dtype=torch.long)
    else:
      x = tokenizer(prompt).to(device)
  else:
    tokenizer = GPT2Tokenizer.from_pretrained(model_type)
    if prompt == '':
      prompt = '<|endoftext|>'
    encoded_input = tokenizer(prompt, return_tensors = 'pt').to(device)
    x = encoded_input['input_ids']

  x = x.expand(num_samples, -1)

  y = model.generate(x, max_new_tokens=steps, do_sample=do_sample, top_k = 40)

  for i in range(num_samples):
    out = tokenizer.decode(y[i].cpu().squeeze())
    print('-'*80)
    print(out)

In [ ]:
generate(prompt='...', num_samples=10, steps=20)